In [1]:
import numpy as np 
import pandas as pd 
import os
import glob
import cv2
import random
import matplotlib.pyplot as plt
import matplotlib.image as mpimg


from keras.models import Sequential
from keras.optimizers import RMSprop
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras import optimizers
from keras.utils import to_categorical, plot_model
from keras.layers import Dense, Input, Conv2D, Flatten, MaxPooling2D, Activation
from keras.models import Model, Sequential
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam
from keras.applications.vgg16 import VGG16
from keras.models import load_model
from keras.utils import np_utils
from keras import backend as K

from keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf

In [2]:
print(os.listdir("Fishes"))
np.random.seed(1)

['Catla', 'Cyprinus_carpio', 'Grass_Carp', 'Silver']


In [3]:
PATH = "Fishes"
os.listdir(PATH)

['Catla', 'Cyprinus_carpio', 'Grass_Carp', 'Silver']

In [4]:
# all image path of parasitized
paths_catla=glob.glob(os.path.join(PATH,'Catla','*.jpg'))
paths_carpio=glob.glob(os.path.join(PATH,'Cyprinus_carpio','*.jpg'))
paths_grass=glob.glob(os.path.join(PATH,'Grass_Carp','*.jpg'))
paths_silver=glob.glob(os.path.join(PATH,'Silver','*.jpg'))
#paths_para=glob.glob(os.path.join(PATH,'Parasitized','*.png'))
#paths_uninf=glob.glob(os.path.join(PATH,'Uninfected','*.png'))
len(paths_catla), len(paths_carpio),len(paths_grass),len(paths_silver)

(49, 152, 36, 57)

In [5]:
path=paths_catla[0]
img=cv2.imread(path) # cv2 stores image in BGR format
img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB) # transforming to RGB format
img.shape

(3456, 5184, 3)

In [6]:
def get_data(path, resize_dim=299):
    img=cv2.imread(path)
    img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    
    #img = cv2.resize(img, dim, interpolation = cv2.INTER_AREA) 
    #img_rgb = np.asarray(np.dstack((img, img, img)), dtype=np.uint8)
    
    img_rz=cv2.resize(img,(resize_dim,resize_dim),interpolation=cv2.INTER_AREA)
    return img_rz

In [7]:
catla_images=np.array([get_data(path) for path in paths_catla])
catla_images.shape

(49, 299, 299, 3)

In [8]:
carpio_images=np.array([get_data(path) for path in paths_carpio])
carpio_images.shape

(152, 299, 299, 3)

In [9]:
grass_images=np.array([get_data(path) for path in paths_grass])
grass_images.shape

(36, 299, 299, 3)

In [10]:
silver_images=np.array([get_data(path) for path in paths_silver])
silver_images.shape

(57, 299, 299, 3)

In [11]:
catla_label = np.array([0 for _ in range(len(catla_images))])
carpio_label = np.array([1 for _ in range(len(carpio_images))])
grass_label = np.array([2 for _ in range(len(grass_images))])
silver_label = np.array([3 for _ in range(len(silver_images))])
#dog_label = np.array([4 for _ in range(len(dog_images))])
#elephant_label = np.array([5 for _ in range(len(elephant_images))])

In [12]:
catla_label.shape, carpio_label.shape, grass_label.shape, silver_label.shape
#, dog_label.shape, elephant_label.shape

((49,), (152,), (36,), (57,))

In [13]:
x_all = np.concatenate((catla_images, carpio_images,grass_images,silver_images), axis=0)
y_all = np.concatenate((catla_label, carpio_label,grass_label,silver_label), axis=0)
print(x_all.shape, y_all.shape)

(294, 299, 299, 3) (294,)


In [14]:
x_train, x_dev, y_train, y_dev = train_test_split(x_all, y_all, test_size=0.2, random_state=1)
print(x_train.shape, x_dev.shape, y_train.shape, y_dev.shape)

(235, 299, 299, 3) (59, 299, 299, 3) (235,) (59,)


In [15]:
# only train and test set
np.save("x_train_200_no_val.npy", x_train)
np.save("y_train_200_no_val.npy", y_train)

np.save("x_test_200_no_val.npy", x_dev)
np.save("y_test_200_no_val.npy", y_dev)


In [16]:
x_train = np.load("x_train_200_no_val.npy")
y_train = np.load("y_train_200_no_val.npy")
x_train.shape, y_train.shape

((235, 299, 299, 3), (235,))

In [17]:
x_dev = np.load("x_test_200_no_val.npy")
y_dev = np.load("y_test_200_no_val.npy")
x_dev.shape, y_dev.shape

((59, 299, 299, 3), (59,))

In [18]:
input_shape = (299, 299, 3)

In [19]:
model = None
model = Sequential()

In [20]:
model.add(Conv2D(32, (3, 3), padding='same', input_shape=input_shape, activation='relu'))
model.add(Conv2D(32, (3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(256, (3, 3), padding='same', activation='relu'))
model.add(Conv2D(256, (3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(1))
model.add(Activation('sigmoid'))
model.compile(loss='categorical_crossentropy',
            optimizer="adadelta",
            metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 299, 299, 32)      896       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 299, 299, 32)      9248      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 149, 149, 32)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 149, 149, 64)      18496     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 149, 149, 64)      36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 74, 74, 64)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 74, 74, 128)       7

In [21]:
model.fit(x=x_train, y=y_train, validation_data = (x_dev,y_dev), epochs=4, verbose=1, batch_size=4)

Epoch 1/4
59/59 [==============================] - 83s 1s/step - loss: 0.0000e+00 - accuracy: 0.2736 - val_loss: 0.0000e+00 - val_accuracy: 0.5593
Epoch 2/4
59/59 [==============================] - 79s 1s/step - loss: 0.0000e+00 - accuracy: 0.3849 - val_loss: 0.0000e+00 - val_accuracy: 0.5593
Epoch 3/4
59/59 [==============================] - 78s 1s/step - loss: 0.0000e+00 - accuracy: 0.5040 - val_loss: 0.0000e+00 - val_accuracy: 0.5593
Epoch 4/4
59/59 [==============================] - 81s 1s/step - loss: 0.0000e+00 - accuracy: 0.5882 - val_loss: 0.0000e+00 - val_accuracy: 0.5593
